# Homo NN MNIST图像分类: 介绍数据集与模型自定义

在该版本中 整个nn的架构有很大的调整，nn模块开发了dataset与model_zoo模块，旨在提供数据集和模型的自定义功能，在这个教程中， 我们将会介绍nn的Dataset机制，以及介绍如何进行模型自定义

## Dataset

在 FATE-1.10中，FATE基于pytorch的Dataset类开发一个新的Dataset基类，用户可以基于这个类，实现\_\_getitem\_\_, \_\_len\_\_, load接口，
load接口接收一个地址，可以从本地读取数据。当提交任务时，可以通过reader组件输入数据地址，HomoNN会使用用户指定的Dataset，使用load接口读取数据，使用数据集进行训练

在FATE-1.10中，提供了table, nlp_tokenizer, image三种数据集，以满足基本的使用需要，但是用户可以随意自定义自己的dataset，dataset的自定义将会在下一教程介绍

### TableDataset
table.py下提供了TableDataset, 用于处理csv格式的数据，并会自动从数据中解析id和label，此处展示部分源代码，可见该数据集的参数使用:

In [ ]:
class TableDataset(Dataset):

    """
     A Table Dataset, load data from a give csv path, or transform FATE DTable

     Parameters
     ----------
     label_col str, name of label column in csv, if None, will automatically take 'y' or 'label' or 'target' as label
     feature_dtype dtype of feature, supports int, long, float, double
     label_dtype: dtype of label, supports int, long, float, double
     label_shape: list or tuple, the shape of label
     flatten_label: bool, flatten extracted label column or not, default is False
     """

    def __init__(self, label_col=None, feature_dtype='float', label_dtype='float', label_shape=None,
                 flatten_label=False):

### TokenizerDataset

nlp_tokenizer.py下提供了TokenizerDataset，基于transformer的BertTokenizer开发，可以从csv里读取字符串，并同时对文本进行自动分词，转换为词id

In [ ]:
class TokenizerDataset(Dataset):
    """
    A Dataset for some basic NLP Tasks, this dataset will automatically transform raw text into word indices
    using BertTokenizer from transformers library,
    see https://huggingface.co/docs/transformers/model_doc/bert?highlight=berttokenizer for details of BertTokenizer

    Parameters
    ----------
    truncation bool, truncate word sequence to 'text_max_length'
    text_max_length int, max length of word sequences
    tokenizer_name_or_path str, name of bert tokenizer(see transformers official for details) or path to local
                                transformer tokenizer folder
    return_label bool, return label or not, this option is for host dataset, when running hetero-NN
    """

    def __init__(self, truncation=True, text_max_length=128,
                 tokenizer_name_or_path="bert-base-uncased",
                 return_label=True):

### ImageDataset
image.py下提供了ImageDataset, 用于简单地处理图像数据，它基于torchvision的ImageFolder开发，可见该数据集的参数使用:

In [ ]:
class ImageDataset(Dataset):

    """

    A basic Image Dataset built on pytorch ImageFolder, supports simple image transform
    Given a folder path, ImageDataset will load images from this folder, images in this
    folder need to be organized in a Torch-ImageFolder format, see
    https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html for details.

    Image name will be automatically taken as the sample id.

    Parameters
    ----------
    center_crop : bool, use center crop transformer
    center_crop_shape: tuple or list
    generate_id_from_file_name: bool, whether to take image name as sample id
    file_suffix: str, default is '.jpg', if generate_id_from_file_name is True, will remove this suffix from file name,
                 result will be the sample id
    return_label: bool, return label or not, this option is for host dataset, when running hetero-NN
    float64: bool, returned image tensors will be transformed to double precision
    label_dtype: str, long, float, or double, the dtype of return label
    """

    def __init__(self, center_crop=False, center_crop_shape=None,
                 generate_id_from_file_name=True, file_suffix='.jpg',
                 return_label=True, float64=False, label_dtype='long'):

本次教程，我们介绍image数据集，它基于pytorch的ImageFolder类开发

### ImageDataset使用
此处 我们用ImageDataset来加载MNIST数据

In [1]:
from federatedml.nn.dataset.image import ImageDataset

In [2]:
# 查看mnist数据集文件夹结构，一共10个文件夹，每个文件夹下放有各个类图像，使用方式与ImageFolder一致
! ls ../examples/data/mnist_folder/

0  1  2  3  4  5  6  7	8  9


In [3]:
dataset = ImageDataset()
dataset.load('../examples/data/mnist/') # 读取mnist数据

In [4]:
len(dataset)

1309

In [5]:
dataset[400] # 查看图像 与ImageFolder使用方式一致

(tensor([[[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0275, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0118, 0.0000, 0.0000,  ...,

## 自定义模型

在 FATE-1.10中，FATE可以通过pipeline提交pytorch Sequential模型.然而，Sequential结合pytorch自带的layer, 还是无法表示更为复杂的模型

因此，FATE-1.10中加入了model_zoo模块，位于federatedml.nn.model_zoo下。现在，你可以定制自己的pytorch模型，**前提是这个模型必须是基于t.nn.Module开发，并实现了forward接口**. 将你实现好的模型文件放入federatedml/nn/model_zoo下, 在提交任务时通过接口指定该模块，homo-nn会自动搜寻并导入你实现的模型

这里，为了完成MNIST分类任务，我们先本地编写一个带有卷积网络的NN模块:

In [6]:
from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)

In [7]:
img_model = ImgNet(10)
img_model

ImgNet(
  (seq): Sequential(
    (0): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1))
    (3): AvgPool2d(kernel_size=3, stride=3, padding=0)
  )
  (fc): Sequential(
    (0): Linear(in_features=48, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)

将模型代码命名为image_net.py，可以直接把它放在federatedml/nn/model_zoo下

或者使用jupyter notebook的快捷接口 直接将其保存到federatedml/nn/model_zoo

In [9]:
from pipeline.component.nn import save_to_fate

## jupyter保存接口

In [ ]:
%%save_to_fate model image_net.py
from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)

In [9]:
! cat ../fate/python/federatedml/nn/model_zoo/image_net.py

from torch import nn
import torch as t
from torch.nn import functional as F

class ImgNet(nn.Module):
    def __init__(self, class_num=10):
        super(ImgNet, self).__init__()
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )
        
        self.fc = t.nn.Sequential(
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, class_num)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        if self.training:
            return x
        else:
            return self.softmax(x)


## 进行本地测试

我们可以先不急着提交任务，可以使用Trainer进行本地测试。上个教程提到，fate自带一个FedAVGTrainer

我们可以使用我们的数据集，自定义的模型，和Trainer进行本地调试，测试是否能够跑通程序
**本地测试的情况下，会跳过所有联邦流程，模型不会进行fed averaging**

In [10]:
from federatedml.nn.homo.trainer.fedavg_trainer import FedAVGTrainer
trainer = FedAVGTrainer(epochs=3, batch_size=256, shuffle=True, data_loader_worker=8) # 参数
trainer.set_model(img_model) # 设置自定义模型

In [11]:
trainer.local_mode() # !! 请务必启用local_mode以跳过联邦流程 !!

In [12]:
optimizer = t.optim.Adam(img_model.parameters(), lr=0.01)
loss = t.nn.CrossEntropyLoss()
trainer.train(train_set=dataset,optimizer=optimizer, loss=loss)

epoch is 0
epoch loss is 2.6494100991781657
epoch is 1
epoch loss is 2.1795332801533256
epoch is 2
epoch loss is 1.5566089883662981


可以跑通！那我们可以提交联邦任务了

## 提交使用自定义模型的Homo-NN任务

In [13]:
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HomoNN
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model

fate_torch_hook(t)


<module 'torch' from '/home/cwj/standalone_fate_install_1.9.0_release/env/python/venv/lib/python3.8/site-packages/torch/__init__.py'>

In [14]:
import os
# 绑定地址到fate name&namespace
fate_project_path = os.path.abspath('../')
host_0 = 10000
host_1 = 9999
pipeline = PipeLine().set_initiator(role='host', party_id=host_0).set_roles(host=[host_0, host_1],
                                                                            arbiter=[host_0])

data_0 = {"name": "mnist_host_0", "namespace": "experiment"}
data_1 = {"name": "mnist_host_1", "namespace": "experiment"}

# 为方便，本示例中两个client使用同一份数据集
data_path_0 = fate_project_path + '/examples/data/mnist'
data_path_1 = fate_project_path + '/examples/data/mnist'
pipeline.bind_table(name=data_0['name'], namespace=data_0['namespace'], path=data_path_0)
pipeline.bind_table(name=data_1['name'], namespace=data_1['namespace'], path=data_path_1)

{'namespace': 'experiment', 'table_name': 'mnist_host_1'}

In [15]:
# 定义reader
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='host', party_id=host_0).component_param(table=data_0)
reader_0.get_party_instance(role='host', party_id=host_1).component_param(table=data_1)

In [16]:
from pipeline.component.homo_nn import DatasetParam, TrainerParam  # 数据集的接口
from pipeline.component.nn.backend.torch.cust_model import CustModel  # 自定义模型的接口

model = t.nn.Sequential(
    t.nn.CustModel(module_name='image_net', class_name='ImgNet', class_num=10) 
    # module_name指定自定义模型的模块名，class_name指定你自定义的类, name之外的参数将会被传递到模型的__init__上，比如
    # 此处的class_num
)

nn_component = HomoNN(name='nn_0',
                      model=model, # 模型
                      loss=t.nn.CrossEntropyLoss(),
                      optimizer=t.optim.Adam(model.parameters(), lr=0.01),
                      dataset=DatasetParam(dataset_name='image'),  # 使用image dataset
                      trainer=TrainerParam(trainer_name='fedavg_trainer', epochs=3, batch_size=256, validation_freqs=1),
                      torch_seed=100 # 全局随机种子
                      )

In [17]:
# 添加组件到pipeline，定义数据IO关系，提交即可
pipeline.add_component(reader_0)
pipeline.add_component(nn_component, data=Data(train_data=reader_0.output.data))
pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=nn_component.output.data))

In [18]:
pipeline.compile()
pipeline.fit()

2022-11-14 11:48:02.547 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211141148018430020

2022-11-14 11:48:02.559 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-14 11:48:03.623 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2022-11-14 11:48:04.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-14 11:48:04.663 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2022-11-14 11:48:05.697 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-14 11:48:06.782 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-14 11:48:07.814 | INFO     | pipel

2022-11-14 11:48:42.397 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:39
2022-11-14 11:48:43.548 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:41
2022-11-14 11:48:44.652 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:42
2022-11-14 11:48:45.709 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:43
2022-11-14 11:48:46.795 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:44
2022-11-14 11:48:47.893 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 0:00:45
2022-11-14 11:48:48.931 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component nn_0, time elapse: 

成功了！并且 我们可以用同样的方式，为任务加一个验证集，看看效果

## 提交使用自定义模型的Homo-NN任务 + 验证集

In [ ]:
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HomoNN
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model

fate_torch_hook(t)

import os
# 绑定地址到fate name&namespace
fate_project_path = os.path.abspath('../')
host_0 = 10000
host_1 = 9999
pipeline = PipeLine().set_initiator(role='host', party_id=host_0).set_roles(host=[host_0, host_1],
                                                                            arbiter=[host_0])

validate_data = {"name": "mnist_validate", "namespace": "experiment"}

# 为方便，本示例中两个client使用同一份数据集
data_path_val = fate_project_path + '/examples/data/mnist_val'
pipeline.bind_table(name=validate_data['name'], namespace=validate_data['namespace'], path=data_path_val)

# 定义reader
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='host', party_id=host_0).component_param(table=data_0)
reader_0.get_party_instance(role='host', party_id=host_1).component_param(table=data_1)
# validate data
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role='host', party_id=host_0).component_param(table=validate_data)
reader_1.get_party_instance(role='host', party_id=host_1).component_param(table=validate_data)

from pipeline.component.homo_nn import DatasetParam, TrainerParam  # 数据集的接口
from pipeline.component.nn.backend.torch.cust_model import CustModel  # 自定义模型的接口

model = t.nn.Sequential(
     t.nn.CustModel(module_name='image_net', class_name='ImgNet', class_num=10) 
    # module_name指定自定义模型的模块名，class_name指定你自定义的类, name之外的参数将会被传递到模型的__init__上，比如
    # 此处的class_num
)

nn_component = HomoNN(name='nn_0',
                      model=model, # 模型
                      loss=t.nn.CrossEntropyLoss(),
                      optimizer=t.optim.Adam(model.parameters(), lr=0.01),
                      dataset=DatasetParam(dataset_name='image'),  # 使用image dataset
                      trainer=TrainerParam(trainer_name='fedavg_trainer', epochs=3, batch_size=256, validation_freqs=1),
                      torch_seed=100 # 全局随机种子
                      )

# 添加组件到pipeline，定义数据IO关系，提交即可
pipeline.add_component(reader_0)
pipeline.add_component(reader_1)
pipeline.add_component(nn_component, data=Data(train_data=reader_0.output.data, validate_data=reader_1.output.data))
pipeline.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=nn_component.output.data))
pipeline.compile()
pipeline.fit()

2022-11-14 11:49:07.637 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211141149044102230

2022-11-14 11:49:07.649 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2022-11-14 11:49:08.667 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
2022-11-14 11:49:10.052 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:02
m2022-11-14 11:49:11.098 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-14 11:49:11.099 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-14 11:49:12.155 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-14 11:49:13.194 | INFO     | pipeline.uti